In [1]:
import numpy as np 
import json
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt  
from matplotlib import style 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import folium 

In [65]:
CLIENT_ID = 'JVFR3G1D3VLAAIRWZ5C0NF0IXDIA0YZ05DNFKPF2PAIYVFOB'
CLIENT_SECRET = 'IFX25FM41NF2N3FGPBJRNBRBSYSNPVLPTRD2JD0RQXG5DZUA' 
VERSION = '20180604'
LIMIT = 300
radius = 500


In [63]:
def get_latlng(neigh):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Virginia Beach, Virginia, United States'.format(neigh))
        lat_lng_coords = g.latlng
    return lat_lng_coords


In [67]:
vbneigh = {'Neighborhoods': ['Alanton', 'Aragona Village', 'Bay Colony', 'Bayside', 'Cape Henry', 'Chesapeake Beach', 'Croatan Beach','Great Neck Point', 'Green Run', 'Kempsville', 'Lago Mar', 'Larkspur', 'London Bridge', 'Lynnhaven', 'Newtown', 'The North End', 'Oceana', 'Ocean Park', 'Pembroke Manor', 'Princess Anne', 'Pungo', 'Red Mill Commons', 'Sandbridge', 'Thalia', 'Thoroughgood']}
vbneighdf= pd.DataFrame(vbneigh)
coordsvb = [ get_latlng(neigh) for neigh in vbneighdf['Neighborhoods']]
dfvbc = pd.DataFrame(coordsvb, columns=['Latitude', 'Longitude'])
vbneighdf['Latitude'] = dfvbc['Latitude']
vbneighdf['Longitude'] = dfvbc['Longitude']
print(vbneighdf.shape)
vbneighdf.head(25)

(25, 3)


,Neighborhoods,Latitude,Longitude
0,Alanton,36.882650,-76.021880
1,Aragona Village,36.848770,-76.149380
2,Bay Colony,36.878110,-76.006830
3,Bayside,36.902120,-76.133070
4,Cape Henry,36.922588,-76.000312
5,Chesapeake Beach,36.915330,-76.121120
6,Croatan Beach,36.844540,-75.973250
7,Great Neck Point,36.900700,-76.086600
8,Green Run,36.798330,-76.091940
9,Kempsville,36.826810,-76.160210


In [66]:
address = 'Virginia Beach, Virginia, United States'
geolocator = Nominatim(user_agent="foursquare-agent")
location = geolocator.geocode(address)
latitudevb = location.latitude
longitudevb = location.longitude
print('The coordinated of VB {}, {}.'.format(latitudevb, longitudevb))
mapvb = folium.Map(location=[latitudevb, longitudevb], zoom_start=11)
for lat, lng, neighborhood in zip(vbneighdf['Latitude'],  vbneighdf['Longitude'], vbneighdf['Neighborhoods']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='#17d44a',fill=True,fill_color='#54de79',fill_opacity=0.7).add_to(mapvb)
mapvb

The coordinated of VB 36.8529841, -75.9774183.


In [5]:
om={'Name':['Aksarben/Elmwood Park'
,'Aksarben Village',
'Applewood',
'Baker Place',
'Bemis Park',
'Bennington',
'Benson',
'Bent Creek',
'Blackstone',
'Boys Town',
'Briar Hills',
'Bridlewood',
'Brookhaven',
'Buena Vista',
'Burlington Road',
'Burnt District',
'Cathedral Landmark District',
'Carter Lake',
'Chalco',
'Conestoga Place',
'Country Club',
'Dahlman',
'Deer Park',
'Dundee',
'Eagle Ridge',
'Elkhorn',
'Elmwood Park',
'Fairacres',
'Florence',
'Field Club',
'First National Business Park',
'Fountain Hills',
'Gifford Park',
'Gold Coast',
'Hanscom Park',
'Happy Hollow',
'Hillsborough',
'Howard Street Apartment District',
'Huntington Park',
'Irvington',
'Keystone',
'Kountze Place',
'La Vista',
'Leavenworth',
'Little Bohemia',
'Little Italy',
'Maple Village',
'Meridian Park',
'Midtown Crossing',
'Millard',
'Miller Park',
'Minne Lusa',
'Morton Meadows',
'Mockingbird Heights',
'Near North Side',
'Oak Heights',
'Old Gold Coast',
'Old Market Historic District',
'Omaha View',
'Orchard Hill',
'Original Montclair (Omaha)',
'Papillion',
'Park Avenue',
'Park East',
'Pepperwood',
'Prospect Hill',
'Ralston',
'Raven Oaks',
'Roanoke',
'Robin Hill',
'Royalwood Estates',
'Regency',
'Roxbury',
'Saratoga',
'Sheelytown',
'South Omaha Main Street Historic District',
'Sporting District',
'Spring Lake',
"Squatter's Row",
'Sunny Slope',
'Trendwood',
'Valley',
'Vinton Street Commercial Historic District','Walnut Hill','Waterloo','Westgate','Westside']}
omneigh=pd.DataFrame(om)
omneigh.dropna(inplace=True)
omneigh.head(88)

,Name
0,Aksarben/Elmwood Park
1,Aksarben Village
2,Applewood
3,Baker Place
4,Bemis Park
...,...
82,Vinton Street Commercial Historic District
83,Walnut Hill
84,Waterloo
85,Westgate


In [6]:
def get_latlngom(neigh):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Omaha, Nebraska, United States'.format(neigh))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude =lat_lng_coords[1]
    return lat_lng_coords
coordsom = [get_latlngom(neigh) for neigh in omneigh['Name']]
dfomc = pd.DataFrame(coordsom, columns=['Latitude', 'Longitude'])
omneigh['Latitude'] = dfomc['Latitude']
omneigh['Longitude'] = dfomc['Longitude']


In [7]:
omneigh

,Name,Latitude,Longitude
0,Aksarben/Elmwood Park,41.244640,-96.001850
1,Aksarben Village,41.238940,-96.013050
2,Applewood,41.291367,-96.059183
3,Baker Place,41.329601,-96.079162
4,Bemis Park,41.269970,-95.968270
...,...,...,...
82,Vinton Street Commercial Historic District,41.229189,-96.206194
83,Walnut Hill,41.270490,-95.976100
84,Waterloo,41.287950,-96.289090
85,Westgate,41.226310,-96.032685


In [8]:
geolocator = Nominatim(user_agent="foursquare-agent")
locationom = geolocator.geocode(address)
latitudeom = locationom.latitude
longitudeom = locationom.longitude
print('The coordinates of Omaha {}, {}.'.format(latitudeom, longitudeom))
mapom = folium.Map(location=[latitudeom, longitudeom], zoom_start=11)
for lat, lng, neighborhood in zip(omneigh['Latitude'], omneigh['Longitude'],omneigh['Name']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='#f5c951',fill=True,fill_color='#edd28a',fill_opacity=0.7).add_to(mapom)
mapom

The coordinates of Omaha 36.8529841, -75.9774183.


In [9]:
neighlatvb = vbneighdf.loc[0, 'Latitude'] 
neighlongvb = vbneighdf.loc[0, 'Longitude'] 
neighnamevb = vbneighdf.loc[0, 'Neighborhoods']
neighlatom = omneigh.loc[0, 'Latitude'] 
neighlongom = omneigh.loc[0, 'Longitude'] 
neighnameom = omneigh.loc[0, 'Name']

In [10]:
urlvb ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighlatvb, neighlongvb, VERSION, radius, LIMIT)
urlom ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighlatom, neighlongom, VERSION, radius, LIMIT)
resultsvb = requests.get(urlvb).json()
resultsvb

{'meta': {'code': 200, 'requestId': '5fbec25d3ede4f01f8dae08d'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Alanton',
  'headerFullLocation': 'Alanton, Virginia Beach',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 36.88715000450007,
    'lng': -76.01626455939116},
   'sw': {'lat': 36.87814999550007, 'lng': -76.02749544060875}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57b51265498ed849ffa879a3',
       'name': 'Apeshyt Hambone',
       'location': {'address': 'Great Neck',
        'lat': 36.884174,
        'lng': -76.022644,
        'labeledLatLngs': [{'label': 'display',
      

In [11]:
resultsom = requests.get(urlom).json()
resultsom

{'meta': {'code': 200, 'requestId': '5fbec25da1e37a62baf556d2'},
 'response': {'headerLocation': 'Aksarben - Elmwood Park',
  'headerFullLocation': 'Aksarben - Elmwood Park, Omaha',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 41.24914000450006,
    'lng': -95.99587634179906},
   'sw': {'lat': 41.24013999550006, 'lng': -96.00782365820092}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52e5be5d11d2e1f4a93adf07',
       'name': 'Tracks Lounge',
       'location': {'address': '1506 S 60th St',
        'lat': 41.24470837404333,
        'lng': -96.00492908044346,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.24470837404333,
          'lng': -96.00492908044346},
         {'label': 'entrance', '

In [12]:
def getcata(row):
    try:
        categorlist = row['categories']
    except:
        categorlist = row['venue.categories']
        
    if len(categorlist) == 0:
        return None
    else:
        return categorlist[0]['name']

In [13]:
venuesvb = resultsvb['response']['groups'][0]['items']
venuesom = resultsom['response']['groups'][0]['items']
nearvenuesvb = json_normalize(venuesvb)
nearvenuesom = json_normalize(venuesom)
filtcolumns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearvenuesvb=nearvenuesvb.loc[:,filtcolumns]
nearvenuesvb['venue.categories'] = nearvenuesvb.apply(getcata, axis=1)
nearvenuesvb.columns = [col.split(".")[-1] for col in nearvenuesvb.columns]
nearvenuesvb.head()
nearvenuesom =nearvenuesom.loc[:, filtcolumns]
nearvenuesom['venue.categories'] = nearvenuesom.apply(getcata, axis=1)
nearvenuesom.columns = [col.split(".")[-1] for col in nearvenuesom.columns]
nearvenuesom.head(90)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Tracks Lounge,Bar,41.244708,-96.004929
1,Jonesy's Taco House,Taco Place,41.244855,-96.005167
2,Boyd & Charlies BBQ,BBQ Joint,41.245421,-96.005196
3,The Gentelman's Room,Historic Site,41.242223,-96.000026
4,Take 1 Talent,Concert Hall,41.248665,-96.003366


In [14]:
nearvenuesvb.head()

,name,categories,lat,lng
0,Apeshyt Hambone,Comedy Club,36.884174,-76.022644


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venuelist=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,  CLIENT_SECRET, VERSION, lat,  lng, radius, LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venuelist.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],  v['venue']['categories'][0]['name']) for v in results])
    nearbyvenue = pd.DataFrame([item for venue_list in venuelist for item in venue_list])
    nearbyvenue.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude',  'Venue Category']   
    return(nearbyvenue)

In [16]:
vbvenues = getNearbyVenues(names=vbneighdf['Neighborhoods'],latitudes=vbneighdf['Latitude'],longitudes=vbneighdf['Longitude'])

Alanton
Aragona Village
Bay Colony
Bayside
Cape Henry
Chesapeake Beach
Croatan Beach
Great Neck Point
Green Run
Kempsville
Lago Mar
Larkspur
London Bridge
Lynnhaven
Newtown
The North End
Oceana
Ocean Park
Pembroke Manor
Princess Anne
Pungo
Red Mill Commons
Sandbridge
Thalia
Thoroughgood


In [17]:
omvenues = getNearbyVenues(names=omneigh['Name'],latitudes=omneigh['Latitude'],longitudes=omneigh['Longitude'])

Aksarben/Elmwood Park
Aksarben Village
Applewood
Baker Place
Bemis Park
Bennington
Benson
Bent Creek
Blackstone
Boys Town
Briar Hills
Bridlewood
Brookhaven
Buena Vista
Burlington Road
Burnt District
Cathedral Landmark District
Carter Lake
Chalco
Conestoga Place
Country Club
Dahlman
Deer Park
Dundee
Eagle Ridge
Elkhorn
Elmwood Park
Fairacres
Florence
Field Club
First National Business Park
Fountain Hills
Gifford Park
Gold Coast
Hanscom Park
Happy Hollow
Hillsborough
Howard Street Apartment District
Huntington Park
Irvington
Keystone
Kountze Place
La Vista
Leavenworth
Little Bohemia
Little Italy
Maple Village
Meridian Park
Midtown Crossing
Millard
Miller Park
Minne Lusa
Morton Meadows
Mockingbird Heights
Near North Side
Oak Heights
Old Gold Coast
Old Market Historic District
Omaha View
Orchard Hill
Original Montclair (Omaha)
Papillion
Park Avenue
Park East
Pepperwood
Prospect Hill
Ralston
Raven Oaks
Roanoke
Robin Hill
Royalwood Estates
Regency
Roxbury
Saratoga
Sheelytown
South Omaha Main

In [18]:
print(vbvenues.shape)
vbvenues.head(3425)

(274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alanton,36.88265,-76.02188,Apeshyt Hambone,36.884174,-76.022644,Comedy Club
1,Aragona Village,36.84877,-76.14938,Iceland Family Skating Center,36.844598,-76.147351,Skating Rink
2,Aragona Village,36.84877,-76.14938,U-Haul Moving & Storage at Town Center,36.844657,-76.148358,Storage Facility
3,Bay Colony,36.87811,-76.00683,Linkhorn Bay,36.874997,-76.007890,Harbor / Marina
4,Bay Colony,36.87811,-76.00683,eat.,36.876320,-76.002856,American Restaurant
...,...,...,...,...,...,...,...
269,Thalia,36.84275,-76.12108,Mark Gonsenhausers Rug & Carpet Superstore,36.842289,-76.118362,Furniture / Home Store
270,Thalia,36.84275,-76.12108,GEICO Insurance Agent,36.842786,-76.125721,Insurance Office
271,Thalia,36.84275,-76.12108,Sprint Store,36.842786,-76.125867,Mobile Phone Shop
272,Thalia,36.84275,-76.12108,Lowenthal,36.842029,-76.116370,Boutique


In [19]:
print(omvenues.shape)
omvenues.head(235)

(1067, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aksarben/Elmwood Park,41.24464,-96.00185,Tracks Lounge,41.244708,-96.004929,Bar
1,Aksarben/Elmwood Park,41.24464,-96.00185,Jonesy's Taco House,41.244855,-96.005167,Taco Place
2,Aksarben/Elmwood Park,41.24464,-96.00185,Boyd & Charlies BBQ,41.245421,-96.005196,BBQ Joint
3,Aksarben/Elmwood Park,41.24464,-96.00185,The Gentelman's Room,41.242223,-96.000026,Historic Site
4,Aksarben/Elmwood Park,41.24464,-96.00185,Take 1 Talent,41.248665,-96.003366,Concert Hall
...,...,...,...,...,...,...,...
230,Conestoga Place,41.27578,-95.94542,"Elemental Scientific, Inc.",41.273436,-95.947660,Paper / Office Supplies Store
231,Conestoga Place,41.27578,-95.94542,The Union For Contemporary Arts,41.278995,-95.947775,Art Gallery
232,Country Club,41.28136,-95.99672,Hartland BBQ,41.285275,-95.997680,BBQ Joint
233,Country Club,41.28136,-95.99672,52nd & NW Radial Hwy,41.284450,-95.995055,Intersection


In [20]:
vbvenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alanton,1,1,1,1,1,1
Aragona Village,2,2,2,2,2,2
Bay Colony,3,3,3,3,3,3
Bayside,29,29,29,29,29,29
Cape Henry,6,6,6,6,6,6
Chesapeake Beach,7,7,7,7,7,7
Croatan Beach,39,39,39,39,39,39
Green Run,9,9,9,9,9,9
Kempsville,2,2,2,2,2,2


In [21]:
omvenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aksarben Village,40,40,40,40,40,40
Aksarben/Elmwood Park,5,5,5,5,5,5
Applewood,7,7,7,7,7,7
Baker Place,3,3,3,3,3,3
Bemis Park,12,12,12,12,12,12
...,...,...,...,...,...,...
Vinton Street Commercial Historic District,2,2,2,2,2,2
Walnut Hill,8,8,8,8,8,8
Waterloo,5,5,5,5,5,5


In [22]:
vbonehot = pd.get_dummies(vbvenues[['Venue Category']], prefix="", prefix_sep="")
vbonehot['Neighborhood'] = vbvenues['Neighborhood'] 
fixed_columns = [vbonehot.columns[-1]] + list(vbonehot.columns[:-1])
vbonehot = vbonehot[fixed_columns]
omonehot = pd.get_dummies(omvenues[['Venue Category']], prefix="", prefix_sep="")
omonehot['Neighborhood'] = omvenues['Neighborhood'] 
fixed_columns = [omonehot.columns[-1]] + list(omonehot.columns[:-1])
omonehot = omonehot[fixed_columns]

In [25]:
vbgroup = vbonehot.groupby('Neighborhood').mean().reset_index()
omgroup = omonehot.groupby('Neighborhood').mean().reset_index()

In [41]:
num_top_venues = 5
for hood in vbgroup['Neighborhood']:
    print("----"+hood+"----")
    temp = vbgroup[vbgroup['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alanton----
                     venue  freq
0              Comedy Club   1.0
1  New American Restaurant   0.0
2               Restaurant   0.0
3                   Resort   0.0
4      Rental Car Location   0.0


----Aragona Village----
                 venue  freq
0     Storage Facility   0.5
1         Skating Rink   0.5
2           Restaurant   0.0
3  Rental Car Location   0.0
4            Racetrack   0.0


----Bay Colony----
                 venue  freq
0               Garden  0.33
1  American Restaurant  0.33
2      Harbor / Marina  0.33
3                Motel  0.00
4               Resort  0.00


----Bayside----
               venue  freq
0     Breakfast Spot  0.10
1       Intersection  0.07
2        Video Store  0.07
3  Convenience Store  0.07
4     Discount Store  0.07


----Cape Henry----
        venue  freq
0  Campground  0.50
1  Public Art  0.17
2  Playground  0.17
3       Beach  0.17
4         ATM  0.00


----Chesapeake Beach----
                venue  freq
0           Sur

In [42]:
for hood in omgroup['Neighborhood']:
    print("----"+hood+"----")
    temp = omgroup[omgroup['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aksarben Village----
          venue  freq
0         Hotel  0.08
1     BBQ Joint  0.05
2   Coffee Shop  0.05
3  Cocktail Bar  0.05
4        Lounge  0.05


----Aksarben/Elmwood Park----
           venue  freq
0     Taco Place   0.2
1   Concert Hall   0.2
2      BBQ Joint   0.2
3            Bar   0.2
4  Historic Site   0.2


----Applewood----
                 venue  freq
0           Taco Place  0.14
1          Golf Course  0.14
2  Martial Arts School  0.14
3                 Park  0.14
4          Social Club  0.14


----Baker Place----
                   venue  freq
0             Playground  0.33
1                    Gym  0.33
2           Home Service  0.33
3                    ATM  0.00
4  Performing Arts Venue  0.00


----Bemis Park----
               venue  freq
0       Soccer Field  0.17
1              Diner  0.08
2             Bakery  0.08
3  Convenience Store  0.08
4        Coffee Shop  0.08


----Bennington----
            venue  freq
0          Museum   1.0
1             ATM  

                             venue  freq
0  Southern / Soul Food Restaurant  0.50
1                             Park  0.25
2                         Pharmacy  0.25
3                              ATM  0.00
4            Performing Arts Venue  0.00


----Orchard Hill----
               venue  freq
0             Garden  0.17
1          Bookstore  0.17
2  Electronics Store  0.17
3                Spa  0.17
4            Theater  0.17


----Original Montclair (Omaha)----
          venue  freq
0  Intersection   0.5
1   Planetarium   0.5
2     Pet Store   0.0
3  Noodle House   0.0
4        Office   0.0


----Papillion----
           venue  freq
0           Park  0.38
1  Jewelry Store  0.12
2        Dog Run  0.12
3            Gym  0.12
4    Event Space  0.12


----Park Avenue----
                  venue  freq
0  Fast Food Restaurant  0.06
1                Lounge  0.03
2       Bubble Tea Shop  0.03
3                 Hotel  0.03
4        Sandwich Place  0.03


----Park East----
                    

In [23]:
vbonehot.head(658)

,Neighborhood,ATM,Adult Boutique,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bakery,...,Tex-Mex Restaurant,Thai Restaurant,Theme Park,Thrift / Vintage Store,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,Alanton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aragona Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aragona Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bay Colony,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bay Colony,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,Thalia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270,Thalia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
271,Thalia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272,Thalia,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)   
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoodvenues_sortedvb =pd.DataFrame(columns=columns)
neighborhoodvenues_sortedvb['Neighborhood'] = vbgroup['Neighborhood']
for ind in np.arange(vbgroup.shape[0]):
    neighborhoodvenues_sortedvb.iloc[ind, 1:] = return_most_common_venues(vbgroup.iloc[ind, :], num_top_venues)
neighborhoodvenues_sortedvb.head(36)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alanton,Comedy Club,Yoga Studio,Food,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store
1,Aragona Village,Skating Rink,Storage Facility,Yoga Studio,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
2,Bay Colony,American Restaurant,Harbor / Marina,Garden,Yoga Studio,Fast Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
3,Bayside,Breakfast Spot,Discount Store,Convenience Store,Video Store,Intersection,Asian Restaurant,Pizza Place,Shipping Store,Sandwich Place,Salon / Barbershop
4,Cape Henry,Campground,Public Art,Playground,Beach,Yoga Studio,Fast Food Restaurant,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
5,Chesapeake Beach,Surf Spot,Gift Shop,Convenience Store,Beach,Seafood Restaurant,Italian Restaurant,Diner,Drugstore,Donut Shop,Discount Store
6,Croatan Beach,Hotel,Pizza Place,Ice Cream Shop,Beach,Clothing Store,Seafood Restaurant,Jewelry Store,Sports Bar,Concert Hall,Dessert Shop
7,Green Run,Restaurant,BBQ Joint,Gym / Fitness Center,Ice Cream Shop,Scenic Lookout,Smoke Shop,Bakery,Hookah Bar,Auto Workshop,Convenience Store
8,Kempsville,Salon / Barbershop,Fried Chicken Joint,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner
9,Lago Mar,Fast Food Restaurant,Miscellaneous Shop,Smoothie Shop,Coffee Shop,Breakfast Spot,Supermarket,Food,Shipping Store,Salon / Barbershop,Convenience Store


In [68]:
neighborhoodvenues_sortedom =pd.DataFrame(columns=columns)
neighborhoodvenues_sortedom['Neighborhood'] = omgroup['Neighborhood']
for ind in np.arange(omgroup.shape[0]):
    neighborhoodvenues_sortedom.iloc[ind, 1:] = return_most_common_venues(omgroup.iloc[ind, :], num_top_venues)
neighborhoodvenues_sortedom.head(86)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aksarben Village,Hotel,Lounge,Farmers Market,BBQ Joint,Cocktail Bar,Coffee Shop,Boutique,Taco Place,Sushi Restaurant,Bike Trail
1,Aksarben/Elmwood Park,BBQ Joint,Taco Place,Concert Hall,Historic Site,Bar,Farmers Market,French Restaurant,Football Stadium,Food Truck,Food Court
2,Applewood,Gym,Taco Place,Social Club,Golf Course,Bar,Martial Arts School,Park,Zoo Exhibit,Food Truck,Food Court
3,Baker Place,Gym,Home Service,Playground,Zoo Exhibit,Football Stadium,Food Truck,Food Court,Flower Shop,Fish Market,Financial or Legal Service
4,Bemis Park,Soccer Field,Intersection,Gym / Fitness Center,Convenience Store,Coffee Shop,Park,Brewery,Gas Station,Theater,Bakery
...,...,...,...,...,...,...,...,...,...,...,...
78,Vinton Street Commercial Historic District,Hardware Store,Fast Food Restaurant,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food Court,Flower Shop,Fish Market,Financial or Legal Service
79,Walnut Hill,Intersection,Bakery,Diner,Gas Station,Construction & Landscaping,Soccer Field,Coffee Shop,Gym,Food Truck,Food Court
80,Waterloo,Gym / Fitness Center,Flower Shop,Tex-Mex Restaurant,Baseball Field,Bar,Furniture / Home Store,French Restaurant,Football Stadium,Food Truck,Food Court
81,Westgate,Baseball Field,Event Service,Pet Store,Grocery Store,Zoo Exhibit,Fast Food Restaurant,French Restaurant,Football Stadium,Food Truck,Food Court


In [45]:
clustersvb= 7
clustersom= 8

vbgroup_clustering = vbgroup.drop('Neighborhood', 1)
kmeansvb = KMeans(n_clusters=clustersvb, random_state=0).fit(vbgroup_clustering)
kmeansvb.labels_[0:10] 
neighborhoodvenues_sortedvb.insert(0, 'Cluster Labels', kmeansvb.labels_)
vbmerged = vbneighdf
vbmerged = vbmerged.join(neighborhoodvenues_sortedvb.set_index('Neighborhood'), on='Neighborhoods')
vbmerged.head(20)

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alanton,36.882650,-76.021880,4.0,Comedy Club,Yoga Studio,Food,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store
1,Aragona Village,36.848770,-76.149380,5.0,Skating Rink,Storage Facility,Yoga Studio,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
2,Bay Colony,36.878110,-76.006830,0.0,American Restaurant,Harbor / Marina,Garden,Yoga Studio,Fast Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
3,Bayside,36.902120,-76.133070,0.0,Breakfast Spot,Discount Store,Convenience Store,Video Store,Intersection,Asian Restaurant,Pizza Place,Shipping Store,Sandwich Place,Salon / Barbershop
4,Cape Henry,36.922588,-76.000312,2.0,Campground,Public Art,Playground,Beach,Yoga Studio,Fast Food Restaurant,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop
5,Chesapeake Beach,36.915330,-76.121120,0.0,Surf Spot,Gift Shop,Convenience Store,Beach,Seafood Restaurant,Italian Restaurant,Diner,Drugstore,Donut Shop,Discount Store
6,Croatan Beach,36.844540,-75.973250,0.0,Hotel,Pizza Place,Ice Cream Shop,Beach,Clothing Store,Seafood Restaurant,Jewelry Store,Sports Bar,Concert Hall,Dessert Shop
7,Great Neck Point,36.900700,-76.086600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Green Run,36.798330,-76.091940,0.0,Restaurant,BBQ Joint,Gym / Fitness Center,Ice Cream Shop,Scenic Lookout,Smoke Shop,Bakery,Hookah Bar,Auto Workshop,Convenience Store
9,Kempsville,36.826810,-76.160210,6.0,Salon / Barbershop,Fried Chicken Joint,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner


In [69]:
omgroup_clustering = omgroup.drop('Neighborhood', 1)
kmeansom = KMeans(n_clusters=clustersom, random_state=0).fit(omgroup_clustering)
kmeansom.labels_[0:10] 
neighborhoodvenues_sortedom.insert(0, 'Cluster Labels', kmeansom.labels_)
ommerged = omneigh
ommerged = ommerged.join(neighborhoodvenues_sortedom.set_index('Neighborhood'), on='Name')
ommerged.head(80)

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aksarben/Elmwood Park,41.244640,-96.001850,1.0,BBQ Joint,Taco Place,Concert Hall,Historic Site,Bar,Farmers Market,French Restaurant,Football Stadium,Food Truck,Food Court
1,Aksarben Village,41.238940,-96.013050,0.0,Hotel,Lounge,Farmers Market,BBQ Joint,Cocktail Bar,Coffee Shop,Boutique,Taco Place,Sushi Restaurant,Bike Trail
2,Applewood,41.291367,-96.059183,1.0,Gym,Taco Place,Social Club,Golf Course,Bar,Martial Arts School,Park,Zoo Exhibit,Food Truck,Food Court
3,Baker Place,41.329601,-96.079162,7.0,Gym,Home Service,Playground,Zoo Exhibit,Football Stadium,Food Truck,Food Court,Flower Shop,Fish Market,Financial or Legal Service
4,Bemis Park,41.269970,-95.968270,0.0,Soccer Field,Intersection,Gym / Fitness Center,Convenience Store,Coffee Shop,Park,Brewery,Gas Station,Theater,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,South Omaha Main Street Historic District,41.201533,-96.033327,4.0,Park,Bowling Alley,Dive Bar,Baseball Field,Mexican Restaurant,Outdoors & Recreation,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
76,Sporting District,41.260680,-95.940260,0.0,Sandwich Place,ATM,Hotel,American Restaurant,Gym / Fitness Center,Art Museum,Indie Theater,Café,Chinese Restaurant,Coffee Shop
77,Spring Lake,41.219177,-95.936179,0.0,Pool,Construction & Landscaping,Playground,Spa,Golf Course,Zoo Exhibit,Eye Doctor,Food Truck,Food Court,Flower Shop
78,Squatter's Row,41.260680,-95.940260,0.0,Sandwich Place,ATM,Hotel,American Restaurant,Gym / Fitness Center,Art Museum,Indie Theater,Café,Chinese Restaurant,Coffee Shop


In [48]:
vbmerged.dropna(inplace=True)
mapclustersvb = folium.Map(location=[latitudevb, longitudevb], zoom_start=11)
x = np.arange(clustersvb)
ys = [i + x + (i*x)**2 for i in range(clustersvb)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(vbmerged['Latitude'], vbmerged['Longitude'],vbmerged['Neighborhoods'],vbmerged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(mapclustersvb)
       
mapclustersvb

In [70]:
ommerged.dropna(inplace=True)
mapclustersom = folium.Map(location=[latitudeom, longitudeom], zoom_start=11)
x = np.arange(clustersom)
ys = [i + x + (i*x)**2 for i in range(clustersom)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(ommerged['Latitude'], ommerged['Longitude'],ommerged['Name'],ommerged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(mapclustersom)
       
mapclustersom

In [73]:
vbmerged.to_csv(r'C:\Users\Jewal\Documents\GitHub\Coursera_Capstone\Virginia Beach Merged.csv',index= False)
ommerged.to_csv(r'C:\Users\Jewal\Documents\GitHub\Coursera_Capstone\Omaha Merged.csv',index = False)